# meme solution que le "Extraction_Job_Title_LinkedIn_With_Account 1", sauf qu'ici je clique sur chaque emploi dans la même page, ensuite extraire les données qui se trouve dans la parti gauche (le détail de l'emploi).

    /!\ Attention a finir :
    
    1-Revoir le probleme de la variable Info_sup1 car le résultat du premier emploi est enregistré dans les mauvaises lignes 
    exemple:  
        Hybride Temps plein Manager
        1 001-5 000 employés · Comptabilité
        Découvrez les tendances récentes de recrutement de BDO France. Essai Premium pour 0 EUR
    il va enregistrer chaque info ci-dessous dans une ligne d'un autre emploi, alors que je souhaite
    avoir ces info dans une seule ligne.
    voir exemple concré dans le df
    
    2- Extraire la description de l'offre

In [1]:
# le module webdriver : permet de naviger sur internet (intéragire avec une page via le navigateur)
from selenium import webdriver

# le module By : permet a selenium de trouver un contenu spécifique dans un page web 
from selenium.webdriver.common.by import By

# la librarie WebDriverWait : permet à selenium d'attendre jusqu'à ce que les éléments de la page web s'affiche correctement.
from selenium.webdriver.support.ui import WebDriverWait

# le module expected_conditions : Ce module contient une collection de conditions que WebDriver peut attendre jusqu'à ce qu'elles se 
# réalisent avant de poursuivre l'exécution du code. On va l'utiliser avec WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# le module time : est utilisée pour effectuer des opérations liées au temps. Dans notre cas on souhaite mettre en pause 
# l'exécution de notre programme pendant un certain nombre de secondes.
import time

# le module BeautifulSoup: est utilisée pour extraire et manipuler les données à partir de fichiers HTML et XML. 
from bs4 import BeautifulSoup

# le module pandas: est utilisée pour la manipulation et l'analyse de données (exemple créer un DataFrame...)
import pandas as pd


In [3]:
# Ouvrir le navigateur Firefox: le navigateur va s'ouvrir automatiquement.
browser = webdriver.Firefox()


#  Accédez à la page login de LinkedIn : get, permet de chercher le lien souhaité
url_loging= browser.get('https://www.linkedin.com/login')


# Saisir automatiquement l'adresse mail : 
    # 1- find_element(By.ID..) : il va chercher sur la page web l'élément par l'ID "username":
        #"""faire une capture d'acran du code source a quoi resemble cet élément"""
    # 2- .send_keys : ensuite cette méthode permet de saisir une chaine de caractére dans cet élément. 
                     # Donc il va saisir notre adresse mail dans le champs de saisie.
browser.find_element(By.ID,'username').send_keys('zakariaeelazzaoui@gmail.com')


# Saisir automatiquement le mot de passe : même logique que l'adresse mail.
browser.find_element(By.ID,'password').send_keys('123456@12345Zak')


# Cliquer automatiquement sur le boutton Connexion pour se connecter :
    # CSS_SELECTOR : permet de spécifier quel élément trouver. 
    # Dans ce cas,on va chercher le bouton qui se trouve à l'intérieur d'un élément ayant la classe login__form_action_container
            #"""faire une capture d'acran du code source a quoi resemble cet élément"""
    # la méthode click(): est appelée pour simuler un clic de souris sur cet élément.
browser.find_element(By.CSS_SELECTOR,'.login__form_action_container button').click()





In [5]:
def slow_scroll(element, distance):
    scroll_step = 60
    for _ in range(0, distance, scroll_step):
        browser.execute_script("arguments[0].scrollBy(0, arguments[1]);", element, scroll_step)
        time.sleep(0.01)

job_info =[]

for i in range(0,25,25):
    
    browser.get("https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=France&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=DD&start="+str(i))
    jobs_list_element = browser.find_element(By.CLASS_NAME,'jobs-search-results-list')
    jobs_detail_element = browser.find_element(By.CLASS_NAME, 'jobs-search__job-details--wrapper')
    
    while True:
        slow_scroll(jobs_list_element,4000)
        time.sleep(2)

        try:
            more_button = WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Voir plus')]"))).click()
        except:
            break  

    s = BeautifulSoup(browser.page_source,'html')

    job = s.find_all('div', class_ = 'flex-grow-1 artdeco-entity-lockup__content ember-view')
    date = s.find_all('ul', class_=('job-card-list__footer-wrapper job-card-container__footer-wrapper flex-shrink-zero display-flex t-sans t-12 t-black--light t-normal t-roman'))
    info_compg = s.find_all('li', class_='job-details-jobs-unified-top-card__job-insight')

    for j,t,c in zip(job[0:5],date[0:5],info_compg[0:5]):
        
        WebDriverWait(browser,2).until(EC.element_to_be_clickable((By.XPATH, "//a[@aria-label="+"'"+j.find('strong').text.strip()+"']"))).click()
        
        while True:
            slow_scroll(jobs_detail_element,4000)
            time.sleep(0.01)
            
            try:
                WebDriverWait(browser, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Voir plus')]")))
            
            except:
                break        
        
        job_title = j.find('strong').text.strip()

        compagny = j.find(class_='job-card-container__primary-description').text.strip()

        location = j.find('li').text.strip()
        
        link = j.a['href']

        if t.find('time') is not None:

            date = t.time['datetime']

            duree = t.time.get_text(strip=True)[0:12]
        
        else:
            date ="Aucune date Trouvée."
            duree ="Aucune date Trouvée."          
        
        
        # 
        bloc_job_info = s.find_all('div', class_='relative job-details-jobs-unified-top-card__container--two-pane')
        bloc_job_descp =""

        if c.text is not None:
            info_sup1= c.text.strip()
        else:
            info_sup1 ="Acune info"
            
        job  = {'Job_Title' : job_title, 'Company': compagny,'Location': location, 'Job_posted':date, 'Duration':duree, 'Link':link,'info_sup1':info_sup1}

        job_info.append(job)

df = pd.DataFrame(job_info)

df[['City', 'Region', 'Pays']] = df['Location'].str.split(', ', expand=True)

df['Pays'].fillna(df['City'], inplace=True)
df ['Region'].fillna(df['City'], inplace=True)

df[['Pays', 'Work_Mode']] = df['Pays'].str.split('(', expand=True)

df['Work_Mode'] = df['Work_Mode'].str.replace(')',"",regex=False)
   
df.drop(columns=['Location'], inplace=False)

df['Link'] ='https://www.linkedin.com'+ df['Link']

df = df.reindex(columns=['Job_Title','Company','City','Region','Pays','Work_Mode','Job_posted','Duration', 'Link','info_sup1'])

print(df.shape)

df.head()







(4, 10)


,Job_Title,Company,City,Region,Pays,Work_Mode,Job_posted,Duration,Link,info_sup1
0,Biostatistician H/F,Ividata Life Sciences,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929631620/...,Hybride\n\n \nTemps plein\n\n \nConfirmé
1,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929635193/...,201-500 employés · Fabrication de produits pha...
2,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929634357/...,"Compétences : Analyse statistique, SAS (logici..."
3,Consultant Senior en Quant - Bdo H/F,BDO France,Paris,Île-de-France,France,Hybride,Aucune date Trouvée.,Aucune date Trouvée.,https://www.linkedin.com/jobs/view/3929637057/...,Découvrez les tendances récentes de recrutemen...
